# Training Hybrid Models

## Checking GPU Availability
First, let's see if we have a GPU available. It is highly recommended to utilize a GPU, although it is also possible to run this experiment on the CPU at a much slower rate.

In [2]:
from experiment import *
from data_utils import load_hybrid_data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Current device: " + str(device))

Current device: cuda


## Loading Training Data
Here, we define the paths to the images and their corresponding masks (targets).
The values for normalization are pre-computed from the training data in order to save time.
In case you want to reproduce all results from the paper, you need to run this notebook several times with different amount of training data. Remember that the actual number is 10-fold higher than what the start/stop inidices suggest, since we have every sample in 10 versions with different SNRs.

Because we wanted to compare the effect of using an increasing number of samples for training, we kept the validation data always the same for a fair and meaningful comparison of the results. Notice, (contrary to the training data) we don't apply any augmentation.

In [3]:
train_dir = os.path.join(os.getcwd(), 'data', 'hybrid', 'train')
mask_dir = os.path.join(os.getcwd(), 'data', 'masks', 'train')


# hybrid
normalizer = {
        'norm_mean': [101],
        'norm_std': [26],
        }

start = 0
stop = 1000
#stop = 2000
#stop = 4000
#stop = 8000
train_dataset = load_hybrid_data(train_dir, mask_dir, start, stop, normalizer, augment=True)

# this subset is always reserved as the validation set, regardless of the number of training samples
start = 9000
stop = 10000
val_dataset = load_hybrid_data(train_dir, mask_dir, start, stop, normalizer)

print('Number of training samples: ' + str(len(train_dataset)))
print('Number of validation samples: ' + str(len(val_dataset)))

Number of training samples: 10000
Number of validation samples: 0


## Creating the Architecture
Here, we create the actual architecture of our network, and set it up to be used in the previously determined device (cpu or cuda).

In [4]:
# this will give create the compact architecture...
# model = get_cmp_thermunet().to(torch.device(device))

# ...or you could use the more complex architecture
model = get_lrg_thermunet().to(torch.device(device))

## Inspecting the Architecture
We can have a look at the architecture and check for instance, if the number of parameters is what we expect.


In [5]:
from torchsummary import summary

device = torch.device('cuda' if next(model.parameters()).is_cuda else 'cpu')

summary(model, input_size=(1, 64, 256), device=str(device))

Layer (type:depth-idx)                   Param #
├─ModuleList: 1-1                        --
|    └─UNetConvBlock: 2-1                --
|    |    └─Sequential: 3-1              2,480
|    └─UNetConvBlock: 2-2                --
|    |    └─Sequential: 3-2              13,888
|    └─UNetConvBlock: 2-3                --
|    |    └─Sequential: 3-3              55,424
|    └─UNetConvBlock: 2-4                --
|    |    └─Sequential: 3-4              221,440
|    └─UNetConvBlock: 2-5                --
|    |    └─Sequential: 3-5              885,248
├─ModuleList: 1-2                        --
|    └─UNetUpBlock: 2-6                  --
|    |    └─Sequential: 3-6              32,896
|    |    └─UNetConvBlock: 3-7           442,624
|    └─UNetUpBlock: 2-7                  --
|    |    └─Sequential: 3-8              8,256
|    |    └─UNetConvBlock: 3-9           110,720
|    └─UNetUpBlock: 2-8                  --
|    |    └─Sequential: 3-10             2,080
|    |    └─UNetConvBlock: 3-1

Layer (type:depth-idx)                   Param #
├─ModuleList: 1-1                        --
|    └─UNetConvBlock: 2-1                --
|    |    └─Sequential: 3-1              2,480
|    └─UNetConvBlock: 2-2                --
|    |    └─Sequential: 3-2              13,888
|    └─UNetConvBlock: 2-3                --
|    |    └─Sequential: 3-3              55,424
|    └─UNetConvBlock: 2-4                --
|    |    └─Sequential: 3-4              221,440
|    └─UNetConvBlock: 2-5                --
|    |    └─Sequential: 3-5              885,248
├─ModuleList: 1-2                        --
|    └─UNetUpBlock: 2-6                  --
|    |    └─Sequential: 3-6              32,896
|    |    └─UNetConvBlock: 3-7           442,624
|    └─UNetUpBlock: 2-7                  --
|    |    └─Sequential: 3-8              8,256
|    |    └─UNetConvBlock: 3-9           110,720
|    └─UNetUpBlock: 2-8                  --
|    |    └─Sequential: 3-10             2,080
|    |    └─UNetConvBlock: 3-1

## Training
Finally, we can start the training procedure.
The folder names to store the results are automatically generated and based on a timestamp.
They are always a subfolder of the training data folder.

In case you want to observe some random output samples during training, you can set the visualization_lvl to either 1 (plot validation data output samples) or 2 (plot validation and training data output samples).

In [6]:
epochs = 100
visualization_lvl = 1

train(model, train_dataset, val_dataset, epochs, visualization_lvl)

print("training finished...")  

Successfully created the directory C:\Users\galiger.gergo\Desktop\ThermUNet-master\data\hybrid\train\2022-12-24_12-26-53.827116\models 


C:\Users\galiger.gergo\Desktop\ThermUNet-master\experiment.py:52: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  assert mean is not -1 and std is not -1, 'Normalize transform not detected!'
C:\Users\galiger.gergo\Desktop\ThermUNet-master\experiment.py:52: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  assert mean is not -1 and std is not -1, 'Normalize transform not detected!'
C:\Users\galiger.gergo\Desktop\ThermUNet-master\experiment.py:52: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  assert mean is not -1 and std is not -1, 'Normalize transform not detected!'
C:\Users\galiger.gergo\Desktop\ThermUNet-master\experiment.py:52: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  assert mean is not -1 and std is not -1, 'Normalize transform not detected!'
C:\Users\galiger.gergo\Desktop\ThermUNet-master\experiment.py:52: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  assert mean is not -1 and std is not -1, 'Normalize trans

ValueError: num_samples should be a positive integer value, but got num_samples=0